In [15]:
import shutil
shutil.rmtree('/Users/Sabrina Nourin/AppData/Local/Programs/Python/Python38/FIRE-SMOKE-DATASET/Train/Smoke')
shutil.rmtree('/Users/Sabrina Nourin/AppData/Local/Programs/Python/Python38/FIRE-SMOKE-DATASET/Test/Smoke')

In [16]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import shutil

In [17]:
TRAINING_DIR = "/Users/Sabrina Nourin/AppData/Local/Programs/Python/Python38/FIRE-SMOKE-DATASET/Train"

training_datagen = ImageDataGenerator(rescale=1./255,
zoom_range=0.15,
horizontal_flip=True,
fill_mode='nearest')

VALIDATION_DIR = "/Users/Sabrina Nourin/AppData/Local/Programs/Python/Python38/FIRE-SMOKE-DATASET/Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(224,224),
shuffle = True,
class_mode='categorical',
  batch_size = 128
)

validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(224,224),
class_mode='categorical',
shuffle = True,
  batch_size= 14
)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [22]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

In [23]:
input_tensor = Input(shape=(224, 224, 3))

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [24]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<=0.1099 and logs.get('loss')<=0.1099):
      print('\n\n Reached The Destination!')
      self.model.stop_training = True
callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 14,
    callbacks=[callbacks]
)

Epoch 1/20
14/14 [==============================] - 94s 7s/step - loss: 8.1230 - acc: 0.7573 - val_loss: 0.1348 - val_acc: 0.9643
Epoch 2/20
14/14 [==============================] - 92s 7s/step - loss: 0.2343 - acc: 0.9228 - val_loss: 0.3610 - val_acc: 0.8214
Epoch 3/20
14/14 [==============================] - 86s 6s/step - loss: 0.3977 - acc: 0.8882 - val_loss: 0.1003 - val_acc: 0.9694
Epoch 4/20
14/14 [==============================] - 85s 6s/step - loss: 0.1420 - acc: 0.9432 - val_loss: 0.0896 - val_acc: 0.9643
Epoch 5/20
14/14 [==============================] - 86s 6s/step - loss: 0.2105 - acc: 0.9151 - val_loss: 0.1099 - val_acc: 0.9694
Epoch 6/20
14/14 [==============================] - ETA: 0s - loss: 0.1025 - acc: 0.9671

 Reached The Destination!
14/14 [==============================] - 87s 6s/step - loss: 0.1025 - acc: 0.9671 - val_loss: 0.0974 - val_acc: 0.9694


In [25]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])


class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<=0.1099 and logs.get('loss')<=0.1099):
      print('\n\n Reached The Destination!')
      self.model.stop_training = True
callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 14,
    callbacks=[callbacks]
)
print(len(base_model.layers))

Epoch 1/10
14/14 [==============================] - 121s 9s/step - loss: 0.4025 - acc: 0.8272 - val_loss: 0.0928 - val_acc: 0.9643
Epoch 2/10
14/14 [==============================] - 118s 8s/step - loss: 0.2703 - acc: 0.8774 - val_loss: 0.0955 - val_acc: 0.9694
Epoch 3/10
14/14 [==============================] - 121s 9s/step - loss: 0.2054 - acc: 0.9109 - val_loss: 0.1027 - val_acc: 0.9745
Epoch 4/10
14/14 [==============================] - 120s 9s/step - loss: 0.1913 - acc: 0.9330 - val_loss: 0.1061 - val_acc: 0.9745
Epoch 5/10
14/14 [==============================] - 119s 8s/step - loss: 0.1897 - acc: 0.9414 - val_loss: 0.1153 - val_acc: 0.9745
Epoch 6/10
14/14 [==============================] - 122s 9s/step - loss: 0.1387 - acc: 0.9563 - val_loss: 0.1201 - val_acc: 0.9643
Epoch 7/10
14/14 [==============================] - 124s 9s/step - loss: 0.1461 - acc: 0.9653 - val_loss: 0.1262 - val_acc: 0.9490
Epoch 8/10
14/14 [==============================] - 122s 9s/step - loss: 0.1260 - a

In [27]:
import numpy as np
#from google.colab import files
from keras.preprocessing import image

Using TensorFlow backend.


In [37]:
path = '/Users/Sabrina Nourin/AppData/Local/Programs/Python/Python38/fire_dataset/test_image/fire_4.jpg'
img = image.load_img(path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) /255
classes = model.predict(x)
print(np.argmax(classes[0])==0, max(classes[0]))

True 0.776989
